In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig#, TrainingArguments
from datasets import load_dataset, Dataset, concatenate_datasets
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer

In [2]:
#model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-14B"
#model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
#model_name = "Qwen/Qwen2.5-7B-Instruct-1M"
model_name = "Qwen/Qwen3-14B"

quantization_config = BitsAndBytesConfig(load_in_4bit=True)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             #torch_dtype=torch.bfloat16,
                                             quantization_config=quantization_config,
                                             torch_dtype="auto",##
                                             device_map="auto"
                                            )

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [5]:
prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
)
print(text)

<|im_start|>user
Give me a short introduction to large language model.<|im_end|>
<|im_start|>assistant



In [10]:
prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False # Switches between thinking and non-thinking modes. Default is True.
)
text

'<|im_start|>user\nGive me a short introduction to large language model.<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\n\n'

In [3]:
import pandas as pd
import os

# 指定user文件夹路径
user_folder = "users_txt"  

filename = os.listdir(user_folder)

print(len(filename))

file_path = os.path.join(user_folder, filename[10])

# 读取文件内容
with open(file_path, 'r', encoding='utf-8') as f:
    content = f.read()

print(content)

21403
(113.374, 22.948) 广州·黄编村, 共4次发帖
2024-11-30 18:09:00, #一条plog告别十一月# 广州·黄编村
2024-03-28 03:50:00, #今天也要开心##开心开心开心# 完美的一天 广州·黄编村
2024-03-14 03:44:00, #广州[超话]# 广州·GOG Club GOG黄源的微博视频
2018-12-22 22:04:00, 广州·市桥街区，节日快乐！喝酒了 广州·市桥街区 一片丹心528729的秒拍视频

(105.232, 28.303) 宜宾·古宋镇, 共1次发帖
2018-04-21 01:51:00, 宜宾·古宋镇，好想你#今日贴纸打卡# 宜宾·古宋镇

(105.237, 28.302) 未知位置, 共1次发帖
2024-01-31 23:44:00, #正能量、励志、经典语录合集[超话]#最好的爱是不打扰，也不放弃，给自己一份尊严，给对方一个尊重。理智的爱一个人，适可而止的纠缠，不远不近的距离，浓淡相宜的往来。情出自愿，事过无悔，不谈亏欠，不负遇见。如果你遇到一个特别喜欢的人，却又明白不能在一起，一定要提醒自己，只做朋友，只谈笑风生，不可动情，不

(113.352, 22.982) 广州·鲍馬爺·首创鲍鱼鸡(长隆店), 共1次发帖
2024-02-27 19:29:00, #一张图证明我来过这里# 广州·鲍馬爺·首创鲍鱼鸡(长隆店)

(113.369, 22.950) 广州·番禺, 共1次发帖
2023-10-17 03:51:00, 无聊的日子 广州·番禺 GOG黄源的微博视频



In [4]:
with open('prompt.txt', 'r') as file:
    prompt = file.read()

file_path = os.path.join(user_folder, filename[10])
with open(file_path, 'r', encoding='utf-8') as f:
    content = f.read()

def generate_prompt(content):
    begin = "<|im_start|>user\n"
    end = "<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\n\n"
    prompt = ("").join([begin, content, end])
    return prompt

new_text = prompt + '\n\nContents: \n' + content
print(generate_prompt(new_text))

<|im_start|>user
The following is a record of posts made by users on social media, including the date, time, geographic location, and latitude and longitude coordinates of each post.
Your task is to find the user's home, workplace or school, and work duration.
You should judge the type of the user's current location based on time and the content of the post, and determine the user's usual work duration based on the time of location switching. If you don't have enough information, don't jump to conclusions.
Your answer should include detailed location and precise coordinates and time, for example:

The user likely lives in 位置, (longitude1, latitude1).
The user likely works/studies at 位置, (longitude2, latitude2). 

Tip: The location that often appears at midnight is likely to be home. Some users are freelancers and their home is their workplace. Number of home and workplace should be 1. Home and work should be in the same city, usually no more than 20km apart
Make your answer as short as

In [ ]:
output_folder = "output"
os.makedirs(output_folder, exist_ok=True)

i = 1

filename = filename[20:]

for fname in filename:
    # 1) 读取输入
    in_path = os.path.join(user_folder, fname)
    with open(in_path, 'r', encoding='utf-8') as f:
        content = f.read()

    # 2) 构造 prompt
    new_text = prompt + '\n\nContents: \n' + content
    input_prompt = generate_prompt(new_text)
    inputs = tokenizer(input_prompt, return_tensors="pt", truncation=True)#.to("cuda")
    input_tokens = inputs["input_ids"].to("cuda")
    attention_mask = inputs["attention_mask"].to("cuda")

    # 3) 调用模型生成
    with torch.cuda.amp.autocast():
        generation_output = model.generate(
            input_ids=input_tokens,
            attention_mask=attention_mask,
            max_new_tokens=128,
            do_sample=True,
            temperature=0.7,
            top_p=0.8,
            top_k=20,
            repetition_penalty=1.1,
            num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id
        )
    op = tokenizer.decode(generation_output[0], skip_special_tokens=True)

    # 4) 保存到 output 文件夹
    out_fname = os.path.splitext(fname)[0] + "_op.txt"
    out_path = os.path.join(output_folder, out_fname)
    with open(out_path, 'w', encoding='utf-8') as wf:
        wf.write(op)

    print(i)
    i = i+1

/tmp/ipykernel_2809/1379633405.py:22: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
/venv/main/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


1


/tmp/ipykernel_2809/1379633405.py:22: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [6]:
input_prompt = generate_prompt(new_text)
#input_tokens = tokenizer(input_prompt, return_tensors="pt")["input_ids"].to("cuda")
#inputs = tokenizer(input_prompt, return_tensors="pt", padding=True, truncation=True).to("cuda")
inputs = tokenizer(input_prompt, return_tensors="pt", truncation=True).to("cuda")
input_tokens = inputs["input_ids"]
attention_mask = inputs["attention_mask"]
with torch.cuda.amp.autocast():
    generation_output = model.generate(
        input_ids=input_tokens,
        attention_mask=attention_mask,
        max_new_tokens=128,
        do_sample=True,                  # 关键：开启采样
        temperature=0.7,                 # 控制输出随机性，越低越确定
        top_p=0.8,                       # nucleus sampling（累计概率阈值）
        #min_p=0,
        top_k=20,                        # top-k sampling（保留前k个）
        repetition_penalty=1.1,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id
    )
op = tokenizer.decode(generation_output[0], skip_special_tokens=True)
print(op)

/tmp/ipykernel_2470/1949298338.py:7: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
/venv/main/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/venv/main/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


user
The following is a record of posts made by users on social media, including the date, time, geographic location, and latitude and longitude coordinates of each post.
Your task is to find the user's home, workplace or school, and work duration.
You should judge the type of the user's current location based on time and the content of the post, and determine the user's usual work duration based on the time of location switching. If you don't have enough information, don't jump to conclusions.
Your answer should include detailed location and precise coordinates and time, for example:

The user likely lives in 位置, (longitude1, latitude1).
The user likely works/studies at 位置, (longitude2, latitude2). 

Tip: The location that often appears at midnight is likely to be home. Some users are freelancers and their home is their workplace. Number of home and workplace should be 1. Home and work should be in the same city, usually no more than 20km apart
Make your answer as short as possible, a

In [7]:
torch.cuda.empty_cache()

In [9]:
del generation_output, inputs, input_tokens, attention_mask

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from datasets import load_dataset, Dataset, concatenate_datasets
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer

import requests
from PIL import Image
from transformers import MllamaForConditionalGeneration, AutoProcessor

In [2]:
model_name = "meta-llama/Llama-3.2-11B-Vision-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
#model = AutoModelForCausalLM.from_pretrained(model_name,
model = MllamaForConditionalGeneration.from_pretrained(model_name,
                                             #load_in_8bit=True,
                                             torch_dtype=torch.bfloat16,
                                             device_map="auto"
                                            )
#processor = AutoProcessor.from_pretrained(model_name)

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [2]:
model_name = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             #load_in_8bit=True,
                                             torch_dtype=torch.bfloat16,
                                             device_map="auto"
                                            )

tokenizer_config.json:   0%|          | 0.00/3.06k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-000002.safetensors:   0%|          | 0.00/8.67G [00:00<?, ?B/s]

model-00002-of-000002.safetensors:   0%|          | 0.00/7.39G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [3]:
import pandas as pd
import os

# 指定user文件夹路径
user_folder = "users_cleaned"  

# 创建一个字典存储所有df
dfs = {}

# 遍历user文件夹下的文件
for i, filename in enumerate(os.listdir(user_folder)):
    if filename.endswith(".csv"):
        # 读取每个csv文件
        filepath = os.path.join(user_folder, filename)
        dfs[i+1] = pd.read_csv(filepath)
        #print(i+1)
        #print(filename)
        #print(dfs[i+1].head())  # 打印前5行以确认读取成功
else:
    if not dfs:  # 判断dfs字典是否为空
        print("未在user文件夹中找到csv文件。")

print(len(dfs))

6105


In [4]:
with open('prompt_poi.txt', 'r') as file:
    prompt = file.read()

def generate_prompt(content):
    begin = "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n"
    #syst = "<<SYS>> You are a helpful assistant, always answer as helpfully as possible.\n If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.<</SYS>>\n"
    #inst = "Read the following text. Does it mention the Gilbert damping constant of a certain material? If so, list the corresponding material and its Gilbert damping canstant.\n" + content
    syst = "You are a helpful AI assistant, answer the question as short as possible.\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n"
    #inst = "What is the attitude of the following text towards electric vehicles or autonomous driving? Answer with positive, negative or neutral. Also classify it as one of the following categories:\n1: Safety\n2: Price and Cost\n3: Technology maturity and infrastructure\n4: User Experience\n5: Policy and market\n6: Ethical and social impact\n7: Other or cannot be classified\nOnly answer the attitude and the category number, and do not answer anything else.\nText: "+content
    end = "\n<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n"
    prompt = (" ").join([begin, syst, content, end])
    return prompt

print(generate_prompt('Hello'))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>
 You are a helpful AI assistant, answer the question as short as possible.
<|eot_id|><|start_header_id|>user<|end_header_id|>
 Hello 
<|eot_id|><|start_header_id|>assistant<|end_header_id|>



In [5]:
def process_dataframe(df):
    """
    筛选 DataFrame 中坐标在深圳范围内的行，并返回格式化后的字符串。

    Args:
        df: 一个 pandas DataFrame，包含 'coordinate', 'publish_time', 'location', 
           'content', 'img_urls' 等列。

    Returns:
        一个格式化后的字符串，包含符合条件的行的信息。
    """
    # 深圳坐标范围
    lon_min, lon_max = 113.7, 114.5
    lat_min, lat_max = 22.4, 22.8

    output_string = ""

    # 过滤出深圳范围内的行
    filtered_rows = []
    for index, row in df.iterrows():
        try:
            lon, lat = map(float, row['coordinate'].split(','))
            if lon_min <= lon <= lon_max and lat_min <= lat <= lat_max:
                filtered_rows.append(row)
        except (ValueError, TypeError, AttributeError) as e:
            print(f"Error processing row with index {index}: {e}")
            continue


    # 限制最大行数
    filtered_rows = filtered_rows[:25]
    
    # 构造输出字符串
    for row in filtered_rows:
        content = row['content']
        if len(content) > 50:
            content = content[:50]
            
        lon, lat = map(float, row['coordinate'].split(','))
        formatted_coordinate = f"{lon:.3f},{lat:.3f}"
        
        output_string += f"{row['publish_time']}, {row['location']} ({formatted_coordinate}), {content}\n"
    
    return output_string

In [9]:
print(tokenizer.special_tokens_map)


{'bos_token': '<｜begin▁of▁sentence｜>', 'eos_token': '<｜end▁of▁sentence｜>', 'pad_token': '<｜end▁of▁sentence｜>'}


In [16]:
with open('prompt_poi.txt', 'r') as file:
    prompt = file.read()

def generate_prompt(content):
    begin = "<｜begin▁of▁sentence｜><|start_header_id|>system<|end_header_id|>\n"
    #syst = "<<SYS>> You are a helpful assistant, always answer as helpfully as possible.\n If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.<</SYS>>\n"
    #inst = "Read the following text. Does it mention the Gilbert damping constant of a certain material? If so, list the corresponding material and its Gilbert damping canstant.\n" + content
    syst = "You are a helpful AI assistant, answer the question as short as possible.\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n"
    #inst = "What is the attitude of the following text towards electric vehicles or autonomous driving? Answer with positive, negative or neutral. Also classify it as one of the following categories:\n1: Safety\n2: Price and Cost\n3: Technology maturity and infrastructure\n4: User Experience\n5: Policy and market\n6: Ethical and social impact\n7: Other or cannot be classified\nOnly answer the attitude and the category number, and do not answer anything else.\nText: "+content
    end = "\n<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n<think>\n好吧，让我仔细分析一下这个问题。"
    prompt = (" ").join([begin, syst, content, end])
    return prompt

new_text = prompt + '\n\nContents: \n' + process_dataframe(dfs[25])
print(generate_prompt(new_text))

<｜begin▁of▁sentence｜><|start_header_id|>system<|end_header_id|>
 You are a helpful AI assistant, answer the question as short as possible.
<|eot_id|><|start_header_id|>user<|end_header_id|>
 The following is a record of posts made by users on social media, including the date, time, geographic location, and latitude and longitude coordinates of each post.
Your task is to find the user's home, workplace or school, and work duration.
You should judge the type of the user's current location based on time and the content of the post, and determine the user's usual work duration based on the time of location switching. If you don't have enough information, don't jump to conclusions.
Your answer should include detailed location and precise coordinates and time, for example:

The user likely lives in 位置, (longitude1, latitude1).
The user likely works/studies at 位置, (longitude2, latitude2).
The user's work/study duration is from -- am to -- pm. 

Tip: The location that often appears at midnight

In [12]:
input_prompt = generate_prompt(new_text)
#input_tokens = tokenizer(input_prompt, return_tensors="pt")["input_ids"].to("cuda")
#inputs = tokenizer(input_prompt, return_tensors="pt", padding=True, truncation=True).to("cuda")
inputs = tokenizer(input_prompt, return_tensors="pt", truncation=True).to("cuda")
input_tokens = inputs["input_ids"]
attention_mask = inputs["attention_mask"]
with torch.cuda.amp.autocast():
    generation_output = model.generate(
        input_ids=input_tokens,
        attention_mask=attention_mask,
        max_new_tokens=2048,
        do_sample=False,
        repetition_penalty=1.1,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id
    )
op = tokenizer.decode(generation_output[0], skip_special_tokens=True)
print(op)

system
 You are a helpful AI assistant, answer the question as short as possible.
user
 The following is a record of posts made by users on social media, including the date, time, geographic location, and latitude and longitude coordinates of each post.
Your task is to find the user's home, workplace or school, and work duration.
You should judge the type of the user's current location based on time and the content of the post, and determine the user's usual work duration based on the time of location switching. If you don't have enough information, don't jump to conclusions.
Your answer should include detailed location and precise coordinates and time, for example:

The user likely lives in 位置, (longitude1, latitude1).
The user likely works/studies at 位置, (longitude2, latitude2).
The user's work/study duration is from -- am to -- pm. 

Tip: The location that often appears at midnight is likely to be home. Some users are freelancers and their home is their workplace. Number of workplac

In [17]:
input_prompt = generate_prompt(new_text)
#input_tokens = tokenizer(input_prompt, return_tensors="pt")["input_ids"].to("cuda")
#inputs = tokenizer(input_prompt, return_tensors="pt", padding=True, truncation=True).to("cuda")
inputs = tokenizer(input_prompt, return_tensors="pt", truncation=True).to("cuda")
input_tokens = inputs["input_ids"]
attention_mask = inputs["attention_mask"]
with torch.cuda.amp.autocast():
    generation_output = model.generate(
        input_ids=input_tokens,
        attention_mask=attention_mask,
        max_new_tokens=2048,
        do_sample=False,
        repetition_penalty=1.1,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id
    )
op = tokenizer.decode(generation_output[0], skip_special_tokens=True)
print(op)

system
 You are a helpful AI assistant, answer the question as short as possible.
user
 The following is a record of posts made by users on social media, including the date, time, geographic location, and latitude and longitude coordinates of each post.
Your task is to find the user's home, workplace or school, and work duration.
You should judge the type of the user's current location based on time and the content of the post, and determine the user's usual work duration based on the time of location switching. If you don't have enough information, don't jump to conclusions.
Your answer should include detailed location and precise coordinates and time, for example:

The user likely lives in 位置, (longitude1, latitude1).
The user likely works/studies at 位置, (longitude2, latitude2).
The user's work/study duration is from -- am to -- pm. 

Tip: The location that often appears at midnight is likely to be home. Some users are freelancers and their home is their workplace. Number of workplac

In [ ]:
import time

output_dir = "output"  # 指定输出目录

data = []
save_interval = 200

start_time = time.time()  # 记录开始时间

for i in range(24,len(dfs)):
    if i % 1 == 0:
        new_text = prompt + '\n\nContents: \n' + process_dataframe(dfs[i])
        input_prompt = generate_prompt(new_text)
        #input_tokens = tokenizer(input_prompt, return_tensors="pt")["input_ids"].to("cuda")
        #inputs = tokenizer(input_prompt, return_tensors="pt", padding=True, truncation=True).to("cuda")
        inputs = tokenizer(input_prompt, return_tensors="pt", truncation=True).to("cuda")
        input_tokens = inputs["input_ids"]
        attention_mask = inputs["attention_mask"]
        with torch.cuda.amp.autocast():
            generation_output = model.generate(
                input_ids=input_tokens,
                attention_mask=attention_mask,
                max_new_tokens=128,
                do_sample=False,
                repetition_penalty=1.1,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.eos_token_id
            )
        op = tokenizer.decode(generation_output[0], skip_special_tokens=True)
        print(i)

        output_filename = os.path.join(output_dir, f"output_{i}.txt")
        # 创建空文件（如果文件不存在）
        with open(output_filename, "a+", encoding="utf-8") as f:
            pass  # 不需要写入任何内容，只是创建文件
        with open(output_filename, "w", encoding="utf-8") as f:
            f.write(op)

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


24
25
26
27
28
29
30
31


In [21]:
del input_tokens
del attention_mask
del inputs
del generation_output
torch.cuda.empty_cache()

In [28]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [4]:
for key in dfs:
  dfs[key] = dfs[key][dfs[key]['coordinate'] != '0,0']

In [5]:
for i in dfs:
    dfs[i] = dfs[i][~dfs[i]['content'].str.contains('转发')]

In [132]:
with open('prompt.txt', 'r') as file:
    prompt = file.read()

In [16]:
with open('prompt_poi.txt', 'r') as file:
    prompt = file.read()

In [122]:
with open('prompt_img.txt', 'r') as file:
    prompt = file.read()

In [6]:
output_string = ""

for index, row in dfs[10].iloc[0:30].iterrows():
    output_string += f"{row['publish_time']}, {row['location']} ({row['coordinate']}), {row['content']}\n"
    img_url = row['img_urls']
    #output_str = f"{row['publish_time']}, {row['location']} ({row['coordinate']}), {row['content']}"
    #print(output_str)
#print(output_string)

#download_image(img_url, save_path)
new_text = prompt + '\n\nContents: \n' + output_string
#print(new_text)
print(generate_prompt(new_text))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>
 You are a helpful AI assistant, answer the question as short as possible.
<|eot_id|><|start_header_id|>user<|end_header_id|>
 The following is a record of posts made by users on social media, including the date, time, geographic location, and latitude and longitude coordinates of each post.
Your task is to find the user's home, workplace or school, and work duration.
You should judge the type of the user's current location based on time and the content of the post, and determine the user's usual work duration based on the time of location switching. If you don't have enough information, don't jump to conclusions.
Your answer should include detailed location and precise coordinates and time, for example:

The user likely lives in 位置, (longitude1, latitude1).
The user likely works/studies at 位置, (longitude2, latitude2).
The user's work/study duration is from -- am to -- pm. 

Tip: The location that often appears at midnight is 

In [9]:
input_prompt = generate_prompt(new_text)
#input_tokens = tokenizer(input_prompt, return_tensors="pt")["input_ids"].to("cuda")
#inputs = tokenizer(input_prompt, return_tensors="pt", padding=True, truncation=True).to("cuda")
inputs = tokenizer(input_prompt, return_tensors="pt", truncation=True).to("cuda")
input_tokens = inputs["input_ids"]
attention_mask = inputs["attention_mask"]
with torch.cuda.amp.autocast():
    generation_output = model.generate(
        input_ids=input_tokens,
        attention_mask=attention_mask,
        max_new_tokens=256,
        do_sample=False,
        repetition_penalty=1.1,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id
    )
op = tokenizer.decode(generation_output[0], skip_special_tokens=True)
print(op)

system
 You are a helpful AI assistant, answer the question as short as possible.
user
 The following is a record of posts made by users on social media, including the date, time, geographic location, and latitude and longitude coordinates of each post.
Your task is to find the user's home, workplace or school, and work duration.
You should judge the type of the user's current location based on time and the content of the post, and determine the user's usual work duration based on the time of location switching. If you don't have enough information, don't jump to conclusions.
Your answer should include detailed location and precise coordinates and time, for example:

The user likely lives in 位置, (longitude1, latitude1).
The user likely works/studies at 位置, (longitude2, latitude2).
The user's work/study duration is from -- am to -- pm. 

Tip: The location that often appears at midnight is likely to be home. Some users are freelancers and their home is their workplace. Number of workplac

In [ ]:
import pandas as pd
import time

output_dir = "output"  # 指定输出目录

data = []
save_interval = 200

start_time = time.time()  # 记录开始时间

for i in range(1,len(dfs)):
    if i % 1 == 0:
        output_string = ""

        for index, row in dfs[i].iloc[0:30].iterrows():
            output_string += f"{row['publish_time']}, {row['location']} ({row['coordinate']}), {row['content']}\n"
            img_url = row['img_urls']
            #output_str = f"{row['publish_time']}, {row['location']} ({row['coordinate']}), {row['content']}"
            #print(output_str)
        #print(output_string)
        
        #download_image(img_url, save_path)
        new_text = prompt + '\n\nContents: \n' + output_string
        #print(new_text)
        #print(generate_prompt(new_text))
        
        input_prompt = generate_prompt(new_text)
        #input_tokens = tokenizer(input_prompt, return_tensors="pt")["input_ids"].to("cuda")
        #inputs = tokenizer(input_prompt, return_tensors="pt", padding=True, truncation=True).to("cuda")
        inputs = tokenizer(input_prompt, return_tensors="pt", truncation=True).to("cuda")
        input_tokens = inputs["input_ids"]
        attention_mask = inputs["attention_mask"]
        with torch.cuda.amp.autocast():
            generation_output = model.generate(
                input_ids=input_tokens,
                attention_mask=attention_mask,
                max_new_tokens=128,
                do_sample=False,
                repetition_penalty=1.1,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.eos_token_id
            )
        op = tokenizer.decode(generation_output[0], skip_special_tokens=True)
        print(op)

        output_filename = os.path.join(output_dir, f"output_{i}.txt")
        # 创建空文件（如果文件不存在）
        with open(output_filename, "a+", encoding="utf-8") as f:
            pass  # 不需要写入任何内容，只是创建文件
        with open(output_filename, "w", encoding="utf-8") as f:
            f.write(op)
        '''
        inst_index = op.find('assistant\n')
        
        if inst_index != -1:
            #print(text)
            #print(op[inst_index + len('assistant\n'):])
            data.append({"number": i, "text": text, "sentiment": op[inst_index + len('assistant\n'):]})
        else:
            #print("未找到'assistant\n'标记")
            data.append({"number": i, "text": text, "sentiment":""})

        # 每 save_interval 个迭代保存一次
        if i % save_interval == 0:
            df = pd.DataFrame(data)
            df.to_csv("output_ev.csv", encoding = 'utf-8-sig', index=False, mode='a', header=False)  # 追加模式
            data = []
            end_time = time.time()  # 记录结束时间
            elapsed_time = end_time - start_time  # 计算用时
            print(f"已保存到 output_ev.csv，当前迭代次数：{i}，用时：{elapsed_time:.2f} 秒")
    
            start_time = end_time  # 更新开始时间，用于计算下一个周期的用时
        '''
# 最后一次保存
#df = pd.DataFrame(data)
#df.to_csv("output_ev.csv", encoding = 'utf-8-sig', index=False, mode='a', header=False)
#print("已保存到 output_ev.csv")

In [64]:
output_dir = "output"  # 指定输出目录

output_filename = os.path.join(output_dir, f"output_{i}.txt")
# 创建空文件（如果文件不存在）
with open(output_filename, "a+", encoding="utf-8") as f:
    pass  # 不需要写入任何内容，只是创建文件
with open(output_filename, "w", encoding="utf-8") as f:
    f.write(op)

In [25]:
messages = [
    {"role": "system", "content": 'Answer question as short as possible.'},
    {"role": "user", "content": [
        #{"type": "image"},
        {"type": "text", "text": new_text}
    ]}
]
input_text = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(
    #image,
    text = input_text,
    add_special_tokens=False,
    return_tensors="pt"
).to(model.device)

output = model.generate(**inputs, max_new_tokens=256)
print(processor.decode(output[0]))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 29 Oct 2024

Answer question as short as possible.<|eot_id|><|start_header_id|>user<|end_header_id|>

The following is a record of posts published by users on social media, including the date, time, geographic location, and latitude and longitude coordinates of each post. Your task is to find the user's points of interest, including but not limited to the location of the user's home, workplace, school, places they often go to, and the coordinate types of these places. The location should be as detailed as possible, the coordinates must be given, and the province and city should be included as much as possible. You should combine the content of the post to determine the type of the user's current location. If you do not have enough information, do not easily draw conclusions.

Contents: 
2015-03-27 13:51:00, 无 (0,0), 落日余晖，美好整个城市。  我正在使用"城市余晖"封面图，好漂亮，你们都快来试试！网页链接  原图 
2015-04-03

In [84]:
import requests

def download_image(image_url, save_path):
    # 发送HTTP GET请求获取图片内容
    response = requests.get(image_url)

    # 检查请求是否成功
    if response.status_code == 200:
        # 以二进制写入方式打开本地文件
        with open(save_path, 'wb') as file:
            file.write(response.content)
        print(f"Image saved to {save_path}")
    else:
        print(f"Failed to download image. Status code: {response.status_code}")

# 示例用法
#image_url = 'https://ww1.sinaimg.cn/large/486c37b1gw6dbeejrajl1j.jpg'
save_path = 'downloaded_image.jpg'

Image saved to downloaded_image.jpg


In [125]:
image = Image.open(save_path)

messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": new_text}
    ]}
]
input_text = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(
    image,
    text = input_text,
    add_special_tokens=False,
    return_tensors="pt"
).to(model.device)

output = model.generate(**inputs, max_new_tokens=256)
print(processor.decode(output[0]))

<|begin_of_text|><|start_header_id|>user<|end_header_id|>

<|image|>The following is a record of a user posting on social media, which contains coordinate information, but the coordinates are likely to be wrong. Analyze the user's real location and what he was doing at the time based on the text content and pictures. The location should be as detailed as possible, and the coordinates must be given, including the province and city as much as possible.

Contents: 
2010-11-15 10:22:00, 无 (0,0), 刚从西藏拍的，发一张试试。  原图 
<|eot_id|><|start_header_id|>assistant<|end_header_id|>

The user is likely in the city of Lhasa, Tibet, China. The image shows a monk standing on a bridge, with a large number of buildings in the background, which is consistent with the architecture of Lhasa. The coordinates provided, (0,0), are likely incorrect and should be ignored.

The user was likely taking a photo of the monk and the surrounding area, possibly as part of a personal project or to share with others on social

In [134]:
output_string = ""

for index, row in dfs[4].iloc[0:10].iterrows():
    output_string += f"{row['publish_time']}, {row['location']} ({row['coordinate']}), {row['content']}\n"
    #output_str = f"{row['publish_time']}, {row['location']} ({row['coordinate']}), {row['content']}"
    #print(output_str)
#print(output_string)

new_text = prompt + '\n\nContents: \n' + output_string
print(new_text)

Below is a record of a user's social media posts, including the date, time and geolocation of each post. Your task is to reconstruct the user's detailed trajectory throughout the specified period, inferring their likely activities and destinations beyond just the locations where they posted.

Each line of your output should represent a distinct point of interest visited by the user, formatted like this:

YYYY-MM-DD - AM/PM - Point of Interest (Latitude, Longitude), Type (if discernable)

Example Output:

2013-10-29 - AM - 武汉·武汉科技大学 (114.26727, 30.44067), Type: School
2013-11-01 - PM - 民族大道 (114.397436, 30.477216), Type: Trip
2013-11-01 - PM - 武汉 (114.397436, 30.477216) Approximately, Type: Home

Important: Do not simply list the locations of the posts. Utilize the content of the posts, time stamps, and geographical proximity to infer the user's full route and potential stops even if they didn't explicitly post from those locations.

User Profile: 
1. 居住地：
根据用户的帖子内容和标签，我推测用户可能居住在深圳市。他们在

In [135]:
messages = [
    {"role": "user", "content": [
        #{"type": "image"},
        {"type": "text", "text": new_text}
    ]}
]
input_text = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(
    #image,
    text = input_text,
    add_special_tokens=False,
    return_tensors="pt"
).to(model.device)

output = model.generate(**inputs, max_new_tokens=1024)
print(processor.decode(output[0]))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 16 Oct 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Below is a record of a user's social media posts, including the date, time and geolocation of each post. Your task is to reconstruct the user's detailed trajectory throughout the specified period, inferring their likely activities and destinations beyond just the locations where they posted.

Each line of your output should represent a distinct point of interest visited by the user, formatted like this:

YYYY-MM-DD - AM/PM - Point of Interest (Latitude, Longitude), Type (if discernable)

Example Output:

2013-10-29 - AM - 武汉·武汉科技大学 (114.26727, 30.44067), Type: School
2013-11-01 - PM - 民族大道 (114.397436, 30.477216), Type: Trip
2013-11-01 - PM - 武汉 (114.397436, 30.477216) Approximately, Type: Home

Important: Do not simply list the locations of the posts. Utilize the content of the posts, time stamps, and geographic

In [131]:
messages = [
    {"role": "user", "content": [
        #{"type": "image"},
        {"type": "text", "text": new_text}
    ]}
]
input_text = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(
    #image,
    text = input_text,
    add_special_tokens=False,
    return_tensors="pt"
).to(model.device)

output = model.generate(**inputs, max_new_tokens=1024)
print(processor.decode(output[0]))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 16 Oct 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Below is a record of a user's social media posts, including the date, time and geolocation of each post. Your task is to reconstruct the user's detailed trajectory throughout the specified period, inferring their likely activities and destinations beyond just the locations where they posted.

Each line of your output should represent a distinct point of interest visited by the user, formatted like this:

YYYY-MM-DD - AM/PM - Point of Interest (Latitude, Longitude), Type (if discernable)

Example Output:

2013-10-29 - AM - 武汉·武汉科技大学 (114.26727, 30.44067), Type: School
2013-11-01 - PM - 民族大道 (114.397436, 30.477216), Type: Trip
2013-11-01 - PM - 武汉 (114.397436, 30.477216) Approximately, Type: Home

Important: Do not simply list the locations of the posts. Utilize the content of the posts, time stamps, and geographic

In [111]:
messages = [
    {"role": "user", "content": [
        #{"type": "image"},
        {"type": "text", "text": new_text}
    ]}
]
input_text = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(
    #image,
    text = input_text,
    add_special_tokens=False,
    return_tensors="pt"
).to(model.device)

output = model.generate(**inputs, max_new_tokens=512)
print(processor.decode(output[0]))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 16 Oct 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Below is a record of a user's social media posts, including the date, time and geolocation of each post. Your task is to reconstruct the user's detailed trajectory throughout the specified period, inferring their likely activities and destinations beyond just the locations where they posted.

Each line of your output should represent a distinct point of interest visited by the user, formatted like this:

YYYY-MM-DD - AM/PM - Point of Interest (Latitude, Longitude), Type (if discernable)

Example Output:

2013-10-29 - AM - 武汉·武汉科技大学 (114.26727, 30.44067), Type: School
2013-11-01 - PM - 民族大道 (114.397436, 30.477216), Type: Trip
2013-11-01 - PM - 武汉 (114.397436, 30.477216) Approximately, Type: Home

Important: Do not simply list the locations of the posts. Utilize the content of the posts, time stamps, and geographic

In [51]:
output = model.generate(**inputs, max_new_tokens=512)
print(processor.decode(output[0]))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 15 Oct 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Below is a record of a user's social media posts, including the date, time and geolocation of each post. Your task is to reconstruct the user's detailed trajectory throughout the specified period, inferring their likely activities and destinations beyond just the locations where they posted.

Each line of your output should represent a distinct point of interest visited by the user, formatted like this:

YYYY-MM-DD - AM/PM - Point of Interest (Latitude, Longitude), Type (if discernable)

Example Output:

2013-10-29 - AM - 武汉·武汉科技大学 (114.26727, 30.44067), Type: School
2013-11-01 - PM - 民族大道 (114.397436, 30.477216), Type: Trip
2013-11-01 - PM - (114.397436, 30.477216)Approximately, Type: Home

Important: Do not simply list the locations of the posts. Utilize the content of the posts, time stamps, and geographical p